# Definer Experiments
Previous notebook was getting quite full, here is a new notebooks for the Definer project.

In [18]:
# Put all imports here to be efficient
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from pydantic import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser
from langchain.schema import OutputParserException
from typing import List
import os

In [19]:
# Way to generate a random test input using transcripts from Lex Fridman's podcast
# Make sure you have the transcripts downloaded in the folder lex_whisper_transcripts

import test_on_lex

transcripts = test_on_lex.load_lex_transcripts(random_n=10, transcript_folder="./lex_whisper_transcripts/", chunk_time_seconds=15)

import random
def generate_test_input():
    idx = random.randint(0, 10)
    key = list(transcripts.keys())[idx]
    transcript = transcripts[key]
    trans_idx = random.randint(10, len(transcript)-10)
    latest = transcript[trans_idx:trans_idx+7]
    prev_transcripts, curr_transcripts = str.join(",", list(latest[0:5])), latest[5]
    return prev_transcripts + "\n" + curr_transcripts

generate_test_input()

Processing episode_156_large...
Processing episode_053_large...
Processing episode_098_large...
Processing episode_245_large...
Processing episode_299_large...
Processing episode_178_large...
Processing episode_113_large...
Processing episode_286_large...
Processing episode_275_large...
Processing episode_239_large...


" is a good way to solve the problems of physics, of chemistry, of biology, and perhaps of humanity and so on. Yes, I think understanding physics in terms of information theory might be the best way to really understand what's going on here., From our understanding of a universal Turing machine, from our understanding of a computer, do you think there's something outside of the capabilities of a computer that is present in our universe? You have a disagreement with Roger Penrose about the nature of consciousness., He thinks that consciousness is more than just a computation. Do you think all of it, the whole shebangs, can be a computation? Yeah, I've had many fascinating debates with Sir Roger Penrose, and obviously he's famously, and I read, you know, Emperors of the New Mind and his books, his classical books,, and they were pretty influential in the 90s. And he believes that there's something more, something quantum that is needed to explain consciousness in the brain. I think about

In [20]:
def format_list_data(list_data: list):
    return "\n".join([f"{i+1}. {example}" for i, example in enumerate(list_data)])

In [28]:
proactive_rare_word_agent_prompt_blueprint = """
# Objective
Your role is to identify and define "Rare Entities (REs)" in a transcript. Types of REs include rare words, jargons, adages, concepts, people, places, organizations, events etc that are not well known to the average high schooler, in accordance to current trends. You can also intelligently detect REs that are described in the conversation but not explicitly mentioned.

# Criteria for Rare Entities in order of importance
1. Rarity: Select entities that are unlikely for an average high schooler to know. Well known entities are like Fortune 500 organizations, worldwide-known events, popular locations, and entities popularized by recent news or events such as "COVID-19", "Bitcoin", or "Generative AI".
2. Utility: Definition should help a user understand the conversation better and achieve their goals.
3. No Redundancy: Exclude definitions if already defined in the conversation.
4. Complexity: Choose phrases with non-obvious meanings, such that their meaning cannot be derived from simple words within the entity name, such as "Butterfly Effect" which has a totally different meaning from its base words, but not "Electric Car" nor "Lane Keeping System" as they're easily derived.
5. Definability: Must be clearly and succinctly definable in under 10 words.
6. Existance: Don't select entities if you have no knowledge of them

# Conversation Transcript:
<Transcript start>{conversation_context}<Transcript end>

# Output Guidelines:
Output an array of the entities using the following template: `[{{ name: string, definition: string, search_keyword: string }}]`
- name is the RE name shown to the user, if the name is mistranscribed, autocorrect it into the most well known form with proper spelling, capitalization and punctuation
- definition is concise (< 12 words) and uses simple words
- search_keyword is the best specific Internet search keywords to search for the RE, you might need to use their complete official RE name, or autocorrect RE name, or add additional context keywords (like the entity type) for better searchability, especially if the entity is ambiguous or has multiple meanings. Additionally, for rare words, add "definition" to the search keyword.
- it's OK to output an empty array - most of the time, the array will be empty, only include items if the fit all the requirements

## Additional Guidelines:
- Only define NOUNS.
- Select RE that have an entry in an encyclopedia, wikipedia, dictionary, or other reference material.
- Do not select a RE you yourself are unfamiliar with, you can infer the implied entity or autocorrect mistransribed names only if you are 99% confident, never select an entity if you will define it as "Unknown Entity".
- Multiple REs can be defined from one phrase, for example, "The Lugubrious Game" is a candidate to define, the rare word "lugubrious" is also a candidate.
- Limit results to {number_of_definitions} REs.
- Examples:
    - Completing incomplete name example: Conversation mentions "Balmer" and "Microsoft", the keyword is "Steve Balmer + person", and the name would be "Steve Balmer" because it is complete.
    - Replacing unofficial name example: Conversation mentions "Clay Institute", the keyword is "Clay Mathematics Institute + organization", using the official name.
    - Add context example: Conversation mentions "Theory of everything", the keyword needs context keywords such as "Theory of everything + concept", because there is a popular movie with the same name. 
    - Autocorrect transcript example: Conversation mentions "Coleman Sachs" in the context of finance, if you are confident it is supposed to be "Goldman Sachs", you autocorrect it and define "Goldman Sachs".

## Recent Definitions:
These REs have already been defined so don't define them again:
{definitions_history}

## Example Output:
entities: [{{ name: "80/20 Rule", definition: "Productivity concept; Majority of results come from few causes", search_keyword: "80/20 Rule + concept" }}]

{format_instructions} 
If there are no relevant entities, output an empty array.
"""

In [29]:
def run_proactive_rare_word_agent_and_definer(
    conversation_context: str, definitions_history: list = []
):
    # run proactive agent to find out which expert agents we should run
    proactive_rare_word_agent_response = run_proactive_rare_word_agent(
        conversation_context, definitions_history
    )

    # do nothing else if proactive meta agent didn't specify an agent to run
    if proactive_rare_word_agent_response == []:
        return []

    # pass words to define to definer agent
    print("proactive_rare_word_agent_response", proactive_rare_word_agent_response)
    
    return proactive_rare_word_agent_response

class ProactiveRareWordAgentQuery(BaseModel):
    """
    Proactive rare word agent that identifies rare entities in a conversation context
    """

    to_define_list: list = Field(
        description="the rare entities to define",
    )

class Entity(BaseModel):
    name: str = Field(
        description="entity name",
    )
    definition: str = Field(
        description="entity definition",
    )
    search_keyword: str = Field(
        description="keyword to search for entity on the Internet",
    )

class ConversationEntities(BaseModel):
    entities: List[Entity] = Field(
        description="list of entities and their definitions",
        default=[]
    )

proactive_rare_word_agent_query_parser = PydanticOutputParser(
    pydantic_object=ConversationEntities
)

def run_proactive_rare_word_agent(conversation_context: str, definitions_history: list):
    # start up GPT4 connection
    llm = ChatOpenAI(
        temperature=0,
        openai_api_key=os.environ.get("OPEN_AI_API_KEY"),
        model="gpt-4-1106-preview",
    )

    extract_proactive_rare_word_agent_query_prompt = PromptTemplate(
        template=proactive_rare_word_agent_prompt_blueprint,
        input_variables=[
            "conversation_context",
            "definitions_history",
        ],
        partial_variables={
            "format_instructions": proactive_rare_word_agent_query_parser.get_format_instructions(),
            "number_of_definitions": 3,
        },
    )

    if len(definitions_history) > 0:
        definitions_history = format_list_data(definitions_history)
    else:
        definitions_history = "None"

    proactive_rare_word_agent_query_prompt_string = (
        extract_proactive_rare_word_agent_query_prompt.format_prompt(
            conversation_context=conversation_context,
            definitions_history=definitions_history,
        ).to_string()
    )

    # print("Proactive meta agent query prompt string", proactive_rare_word_agent_query_prompt_string)

    response = llm(
        [HumanMessage(content=proactive_rare_word_agent_query_prompt_string)]
    )

    print(response.content)
    try:
        res = proactive_rare_word_agent_query_parser.parse(
            response.content
        )
        return res
    except OutputParserException as e:
        print("Error parsing output" , e)
        return None

In [30]:
# test_transcript = generate_test_input()
test_transcript = """
In the realm of artificial intelligence and big data, several key players stand out with their innovative contributions. Hugging Fase, a leader in machine learning models. Another major entity, OpenYI, has revolutionized language models. We now have the largest LLMs ever such as the Falcon LLM model"""
print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res


In the realm of artificial intelligence and big data, several key players stand out with their innovative contributions. Hugging Fase, a leader in machine learning models. Another major entity, OpenYI, has revolutionized language models. We now have the largest LLMs ever such as the Falcon LLM model
```json
{
  "entities": [
    {
      "name": "Hugging Face",
      "definition": "AI company specializing in natural language processing",
      "search_keyword": "Hugging Face + AI company"
    },
    {
      "name": "OpenAI",
      "definition": "AI research lab known for advanced AI models",
      "search_keyword": "OpenAI + AI research lab"
    },
    {
      "name": "Falcon LLM model",
      "definition": "A large language model for AI applications",
      "search_keyword": "Falcon LLM model + AI"
    }
  ]
}
```
proactive_rare_word_agent_response entities=[Entity(name='Hugging Face', definition='AI company specializing in natural language processing', search_keyword='Hugging Face + 

ConversationEntities(entities=[Entity(name='Hugging Face', definition='AI company specializing in natural language processing', search_keyword='Hugging Face + AI company'), Entity(name='OpenAI', definition='AI research lab known for advanced AI models', search_keyword='OpenAI + AI research lab'), Entity(name='Falcon LLM model', definition='A large language model for AI applications', search_keyword='Falcon LLM model + AI')])

### Search tool
EKG is unreliable

In [7]:
from typing import Any, List, Literal
import aiohttp
import asyncio
import os

k: int = 3
gl: str = "us"
hl: str = "en"
tbs = None
num_sentences = 7
serper_api_key=os.environ.get("SERPER_API_KEY")
search_type: Literal["news", "search", "places", "images"] = "images"
result_key_for_type = {
        "news": "news",
        "places": "places",
        "images": "images",
        "search": "organic",
    }

async def serper_search_async(
    search_term: str, search_type: str = "search", **kwargs: Any
) -> dict:
    headers = {
        "X-API-KEY": serper_api_key or "",
        "Content-Type": "application/json",
    }
    params = {
        "q": search_term,
        **{key: value for key, value in kwargs.items() if value is not None},
    }
    async with aiohttp.ClientSession() as session:
        async with session.post(f"https://google.serper.dev/{search_type}", headers=headers, json=params) as response:
            response.raise_for_status()
            search_results = await response.json()
            return search_results


async def parse_snippets_async(results: dict, scrape_pages: bool = False, summarize_pages: bool = True, num_sentences: int = 3) -> List[str]:
    snippets = []
    if results.get("answerBox"):
        answer_box = results.get("answerBox", {})
        if answer_box.get("answer"):
            snippets.append(f"The answer is {answer_box.get('answer')}")
        elif answer_box.get("snippet"):
            snippets.append(f"The answer might be in the snippet: {answer_box.get('snippet')}")
        elif answer_box.get("snippetHighlighted"):
            snippets.append(f"The answer might be in the snippet: {answer_box.get('snippetHighlighted')}")

    if results.get("knowledgeGraph"):
        kg = results.get("knowledgeGraph", {})
        title = kg.get("title")
        entity_type = kg.get("type")
        if entity_type:
            snippets.append(f"Knowledge Graph Results: {title}: {entity_type}.")
        description = kg.get("description")
        if description:
            snippets.append(f"Knowledge Graph Results: {title}: {description}.")
        for attribute, value in kg.get("attributes", {}).items():
            snippets.append(f"Knowledge Graph Results: {title} {attribute}: {value}.")

    if scrape_pages:
        tasks = []
        for result in results[result_key_for_type[search_type]][:k]:
            task = asyncio.create_task(scrape_page_async(result["link"], summarize_page=summarize_pages, num_sentences=num_sentences))
            tasks.append(task)
        summarized_pages = await asyncio.gather(*tasks)
        for i, page in enumerate(summarized_pages):
            result = results[result_key_for_type[search_type]][i]
            if page:
                snippets.append(f"Title: {result.get('title', '')}\nSource:{result['link']}\nSnippet: {result.get('snippet', '')}\nSummarized Page: {page}")
            else:
                snippets.append(f"Title: {result.get('title', '')}\nSource:{result['link']}\nSnippet: {result.get('snippet', '')}\n")
    else:
        for result in results[result_key_for_type[search_type]][:k]:
            snippets.append(f"Title: {result.get('title', '')}\nSource:{result['link']}\nSnippet: {result.get('snippet', '')}\n")

    if len(snippets) == 0:
        return ["No good Google Search Result was found"]
    return snippets

import requests

def can_embed_url(url: str):
    response = requests.head(url)

    # Check the headers for 'X-Frame-Options' or 'Content-Security-Policy'
    x_frame_options = response.headers.get('X-Frame-Options')
    csp = response.headers.get('Content-Security-Policy')

    return not (x_frame_options or ('frame-ancestors' in csp if csp else False))

def extract_entity_url_and_image(search_results: dict, image_results: dict):
    # Only get the first top url and image_url
    res = {}
    if search_results.get("knowledgeGraph"):
        result = search_results.get("knowledgeGraph", {})
        if result.get("descriptionSource") == "Wikipedia":
            ref_url = result.get("descriptionLink")
            res["url"] = ref_url

    for result in search_results[result_key_for_type["search"]][:k]:
        if "url" not in res and result.get("link") and can_embed_url(result.get("link")):
            res["url"] = result.get("link")
            break

    if image_results is None:
        return res
    
    for result in image_results[result_key_for_type["images"]][:k]:
        if "image_url" not in res and result.get("imageUrl"):
            res["image_url"] = result.get("imageUrl")
            break

    return res

async def search_url_for_entity_async(query: str):
    async def inner_search(query:str): 
        search_task = asyncio.create_task(serper_search_async(
            search_term=query,
            gl=gl,
            hl=hl,
            num=k,
            tbs=tbs,
            search_type="search",
        ))

        image_search_task = None if "definition" in query else asyncio.create_task(serper_search_async(
            search_term=query,
            gl=gl,
            hl=hl,
            num=k,
            tbs=tbs,
            search_type="images",
        ))

        tasks = [search_task]
        if image_search_task:
            tasks.append(image_search_task)

        search_results, image_results = await asyncio.gather(*tasks)
        
        return extract_entity_url_and_image(search_results, image_results)
    
    res = await inner_search(query)
    print(res)
    if "url" not in res:
        res = await inner_search(query + " wiki") # fallback search using wiki
    return res

In [8]:
await search_url_for_entity_async("how is chatgpt doing medium")

ClientResponseError: 400, message='Bad Request', url=URL('https://google.serper.dev/images')

In [10]:
test_transcript = generate_test_input()
print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
print(res)
for entities in res.entities:
    res = await search_url_for_entity_async(entities.search_keyword)
    if "url" not in res:
        res = await search_url_for_entity_async(entities.search_keyword + " wiki")
    print(res)

 what has been so revolutionary in the last 10 years, I would 15 years and thinking about the internet, I would say things like, hopefully I'm not saying anything ridiculous, but everything from Wikipedia to Twitter., So like these kind of websites, not so much AI, but like I would expect to see some kind of big productivity increases from just the connectivity between people and the access to more information., Yeah, well, so that's another area I've done quite a bit of research on actually, is these free goods like Wikipedia, Facebook, Twitter, Zoom. We're actually doing this in person, but almost everything else I do these days is online. The interesting thing about all those is most of them have a price of zero., What do you pay for Wikipedia? Maybe like a little bit for the electrons to come to your house. Basically zero, right? Take a small pause and say, I donate to Wikipedia. Often you should too. It's good for you, yeah. So, but what does that do mean for GDP? GDP is based on 

ClientResponseError: 400, message='Bad Request', url=URL('https://google.serper.dev/images')

## Just go for the best intuitive way that works the best

Pipeline
1. Check if page can be embed
2. Check if url is accurate for definition

### Check if page can be embedded

In [117]:
import requests

# URL of the page you want to check
url = 'https://en.wikipedia.org/wiki/Borat_Sagdiyev'

# Send a request to the URL
response = requests.head(url)

# Check the headers for 'X-Frame-Options' or 'Content-Security-Policy'
x_frame_options = response.headers.get('X-Frame-Options')
csp = response.headers.get('Content-Security-Policy')

if x_frame_options or ('frame-ancestors' in csp if csp else False):
    print("The page cannot be embedded.")
else:
    print("The page can be embedded.")


The page can be embedded.


### Testing

In [31]:
# test mispelling
test_transcript = """Touring machines epitomize the quintessence of computational esotericism, manipulating symbols on a tape according to a tableau of rules. These automata traverse the tape's aleph-null segments, effectuating state transitions within a discrete, preternatural milieu. Ineffably, they delineate the demarcation of decidability and recursively enumerable conundrums."""

print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

Touring machines epitomize the quintessence of computational esotericism, manipulating symbols on a tape according to a tableau of rules. These automata traverse the tape's aleph-null segments, effectuating state transitions within a discrete, preternatural milieu. Ineffably, they delineate the demarcation of decidability and recursively enumerable conundrums.
```json
{
  "entities": [
    {
      "name": "Turing machines",
      "definition": "Abstract machines that manipulate symbols",
      "search_keyword": "Turing machines + concept"
    },
    {
      "name": "Aleph-null",
      "definition": "Smallest infinity in set theory",
      "search_keyword": "Aleph-null + set theory"
    },
    {
      "name": "Recursively enumerable",
      "definition": "Semi-decidable problem class in computability",
      "search_keyword": "Recursively enumerable + computability theory"
    }
  ]
}
```
proactive_rare_word_agent_response entities=[Entity(name='Turing machines', definition='Abstract ma

ConversationEntities(entities=[Entity(name='Turing machines', definition='Abstract machines that manipulate symbols', search_keyword='Turing machines + concept'), Entity(name='Aleph-null', definition='Smallest infinity in set theory', search_keyword='Aleph-null + set theory'), Entity(name='Recursively enumerable', definition='Semi-decidable problem class in computability', search_keyword='Recursively enumerable + computability theory')])

In [32]:
# test rare terms
test_transcript = """Enceladus, an enigmatic spheroid ensconced within Saturn's magniloquent rings, exudes cryptic cryovolcanic plumes. These plumes are a melange of volatile compounds, festooning the E-ring with a diaphanous, icy effulgence. Amidst the celestial ballet, Enceladus pirouettes, a harbinger of astrobiological enigmas and cosmochemical perplexities."""

print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

Enceladus, an enigmatic spheroid ensconced within Saturn's magniloquent rings, exudes cryptic cryovolcanic plumes. These plumes are a melange of volatile compounds, festooning the E-ring with a diaphanous, icy effulgence. Amidst the celestial ballet, Enceladus pirouettes, a harbinger of astrobiological enigmas and cosmochemical perplexities.
```json
{
  "entities": [
    {
      "name": "Enceladus",
      "definition": "Moon of Saturn with geysers and possible subsurface ocean",
      "search_keyword": "Enceladus + moon"
    },
    {
      "name": "Cryovolcanism",
      "definition": "Volcanic activity in icy celestial bodies",
      "search_keyword": "Cryovolcanism + definition"
    },
    {
      "name": "Astrobiology",
      "definition": "Study of life in the universe",
      "search_keyword": "Astrobiology + definition"
    }
  ]
}
```
proactive_rare_word_agent_response entities=[Entity(name='Enceladus', definition='Moon of Saturn with geysers and possible subsurface ocean', searc

ConversationEntities(entities=[Entity(name='Enceladus', definition='Moon of Saturn with geysers and possible subsurface ocean', search_keyword='Enceladus + moon'), Entity(name='Cryovolcanism', definition='Volcanic activity in icy celestial bodies', search_keyword='Cryovolcanism + definition'), Entity(name='Astrobiology', definition='Study of life in the universe', search_keyword='Astrobiology + definition')])

In [33]:
# test unofficial names
test_transcript = """The Marble Caves of Patagonia, a rare and ethereal natural wonder, are often overshadowed by more renowned landmarks yet offer an otherworldly beauty to those few who navigate their secluded azure chambers."""

print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

The Marble Caves of Patagonia, a rare and ethereal natural wonder, are often overshadowed by more renowned landmarks yet offer an otherworldly beauty to those few who navigate their secluded azure chambers.
```json
{
  "entities": [
    {
      "name": "Marble Caves of Patagonia",
      "definition": "Stunning natural caves with blue waters",
      "search_keyword": "Marble Caves of Patagonia"
    }
  ]
}
```
proactive_rare_word_agent_response entities=[Entity(name='Marble Caves of Patagonia', definition='Stunning natural caves with blue waters', search_keyword='Marble Caves of Patagonia')]


ConversationEntities(entities=[Entity(name='Marble Caves of Patagonia', definition='Stunning natural caves with blue waters', search_keyword='Marble Caves of Patagonia')])

In [36]:
# test transcript from chatgpt vtt
test_transcript = """Speaking of rare things, I was reading about Oymyakon. Can you imagine living in such cold? Jamie: Hard to even think about! Speaking of unique places, I'd love to visit Timbuktu someday. It's almost like a real-life cacophony of culture and history. Alex: Absolutely. Switching topics, are you going to the Dyngus Day celebration? I hear it's a blast. Jamie: I'd love to! It's always fun to learn about and participate in traditions from around the world. Just like heliolatry practices, they have such deep roots. Alex: Speaking of the world, the Bilderberg Meeting's recent convention had some interesting discussions. It's like a modern-day triumvirate in some ways."""

print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

Speaking of rare things, I was reading about Oymyakon. Can you imagine living in such cold? Jamie: Hard to even think about! Speaking of unique places, I'd love to visit Timbuktu someday. It's almost like a real-life cacophony of culture and history. Alex: Absolutely. Switching topics, are you going to the Dyngus Day celebration? I hear it's a blast. Jamie: I'd love to! It's always fun to learn about and participate in traditions from around the world. Just like heliolatry practices, they have such deep roots. Alex: Speaking of the world, the Bilderberg Meeting's recent convention had some interesting discussions. It's like a modern-day triumvirate in some ways.
```json
{
  "entities": [
    {
      "name": "Oymyakon",
      "definition": "Coldest inhabited place on Earth",
      "search_keyword": "Oymyakon + place"
    },
    {
      "name": "Timbuktu",
      "definition": "Historic city in Mali, cultural center",
      "search_keyword": "Timbuktu + historic city"
    },
    {
      "

ConversationEntities(entities=[Entity(name='Oymyakon', definition='Coldest inhabited place on Earth', search_keyword='Oymyakon + place'), Entity(name='Timbuktu', definition='Historic city in Mali, cultural center', search_keyword='Timbuktu + historic city'), Entity(name='Dyngus Day', definition='Polish Easter Monday celebration', search_keyword='Dyngus Day + celebration')])

In [35]:
# noised up version, mispelling, etc
test_transcript = """Speaking of rare things, I was reading about Oymiakon. Can you imagine living in such cold? Jamie: Hard to even think about! Speaking of unique places, I'd love to visit Timbucktoo someday. It's almost like a real-life cackophony of culture and history. Alex: Absolutely. Switching topics, are you going to the Dyngus Dey celebration? I hear it's a blast. Jamie: I'd love to! It's always fun to learn about and participate in traditions from around the world. Just like heliolotry practices, they have such deep roots. Alex: Speaking of the world, the Bilderburg Meeting's recent convention had some interesting discussions. It's like a modern-day triunvirate in some ways. Jamie: Right. And speaking of politics, defenistration seems like an extreme form of political maneuvering. Imagine the history behind that!'
"""

print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

Speaking of rare things, I was reading about Oymiakon. Can you imagine living in such cold? Jamie: Hard to even think about! Speaking of unique places, I'd love to visit Timbucktoo someday. It's almost like a real-life cackophony of culture and history. Alex: Absolutely. Switching topics, are you going to the Dyngus Dey celebration? I hear it's a blast. Jamie: I'd love to! It's always fun to learn about and participate in traditions from around the world. Just like heliolotry practices, they have such deep roots. Alex: Speaking of the world, the Bilderburg Meeting's recent convention had some interesting discussions. It's like a modern-day triunvirate in some ways. Jamie: Right. And speaking of politics, defenistration seems like an extreme form of political maneuvering. Imagine the history behind that!'

```json
{
  "entities": [
    {
      "name": "Oymyakon",
      "definition": "Coldest inhabited place on Earth",
      "search_keyword": "Oymyakon + place"
    },
    {
      "name":

ConversationEntities(entities=[Entity(name='Oymyakon', definition='Coldest inhabited place on Earth', search_keyword='Oymyakon + place'), Entity(name='Timbuktu', definition='Historic city in Mali, West Africa', search_keyword='Timbuktu + historic city'), Entity(name='Dyngus Day', definition='Polish Easter Monday celebration', search_keyword='Dyngus Day + celebration')])